In [ ]:
# Full reset of the camera
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
!ls -ltrh /dev/video*

# USB Camera (Logitech C270 webcam)
from jetcam.usb_camera import USBCamera
camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# CSI Camera (Raspberry Pi Camera Module V2)
# from jetcam.csi_camera import CSICamera
# camera = CSICamera(width=224, height=224)

camera.running = True
print("camera created")

In [ ]:
# Set up the data directory location if not there already
import os

DATA_DIR = '/nvdli-nano/data/object-detection/custom/raw_images'
!mkdir -p {DATA_DIR}

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from dataset import ImageCollect

# initialize active dataset
directory = DATA_DIR
dataset = ImageCollect(directory)

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count()

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value)
    count_widget.value = dataset.get_count()
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), count_widget, save_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

In [ ]:
display(data_collection_widget)